In [2]:
from torch_geometric.datasets import Planetoid
import torch
import numpy as np
import networkx as nx

/home/lzq/anaconda3/envs/pyg-cpu/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def get_k_layer_neighbors_feature(node, num_steps, G, num_nodes):
    neighbors = [node]
    current_node = node
    while len(neighbors) <= num_nodes:
        for j in range(num_steps):
            # Get the indices of the current node's neighbors
            neighbor_indices = list(G.neighbors(current_node))
            # Choose a random neighbor
            if len(neighbor_indices) > 0:
                chosen_neighbor = np.random.choice(neighbor_indices)
                # Move to the chosen neighbor
                current_node = chosen_neighbor
            else:
                break
        neighbors.append(current_node)
    return neighbors

In [5]:
# Load the Cora dataset
dataset = Planetoid(root='data', name='Cora')
data = dataset[0]
# Convert the data object to a NetworkX graph
G = nx.Graph()
edge_index = data.edge_index.numpy()
G.add_edges_from(edge_index.T)
# Set the number of steps for the random walk
num_nodes = 3
# Loop over every node and perform a random walk   
for i in range(len(G.nodes)):
    node_features = []
    for num_steps in range(1,4):
        neighbors =  get_k_layer_neighbors_feature(i, num_steps, G, num_nodes)
        num_neighbors = int(len(neighbors))
        # print the neighbros
        print(f"Node {i} has {num_neighbors - 1} neighbors in {num_steps} layer: {neighbors[1:num_neighbors+1]}")
        # add the neighbors
        neighbor_features = data.x[neighbors[1:]]
        temp_feature = data.x[neighbors[0]]
        node_features.append(torch.add(temp_feature, neighbor_features.mean(dim=0))) 
        # concate the feature
    concatenated_features = data.x[i]
    for node_feature in node_features:
        concatenated_features = torch.cat([concatenated_features, node_feature], dim=0)
    print(concatenated_features.shape)

Node 0 has 3 neighbors in 1 layer: [2582, 1166, 1986]
Node 0 has 3 neighbors in 2 layer: [1701, 228, 985]
Node 0 has 3 neighbors in 3 layer: [1701, 1701, 1567]
torch.Size([5732])
Node 1 has 3 neighbors in 1 layer: [2, 332, 665]
Node 1 has 3 neighbors in 2 layer: [470, 809, 1529]
Node 1 has 3 neighbors in 3 layer: [2, 1986, 1709]
torch.Size([5732])
Node 2 has 3 neighbors in 1 layer: [1666, 49, 1666]
Node 2 has 3 neighbors in 2 layer: [654, 652, 857]
Node 2 has 3 neighbors in 3 layer: [531, 2308, 531]
torch.Size([5732])
Node 3 has 3 neighbors in 1 layer: [2544, 3, 2544]
Node 3 has 3 neighbors in 2 layer: [3, 3, 3]
Node 3 has 3 neighbors in 3 layer: [2544, 3, 2544]
torch.Size([5732])
Node 4 has 3 neighbors in 1 layer: [2176, 1256, 2175]
Node 4 has 3 neighbors in 2 layer: [1256, 1256, 1256]
Node 4 has 3 neighbors in 3 layer: [2175, 982, 1761]
torch.Size([5732])
Node 5 has 3 neighbors in 1 layer: [1659, 5, 1629]
Node 5 has 3 neighbors in 2 layer: [5, 466, 952]
Node 5 has 3 neighbors in 3 la